In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In this case study, we try to find best accurancy for voice data which is include 21 columns and 3168 rows aboust voice numeric params. Label column is our target column, and we convert this column to binary. After this encoding, we look for correlation with this target between other columns. Finally, we train and test our model with sklearn ML algoritms. We choose min error in eeach model perform and choose one of them which is best accurancy.

* K-Nearest Neighbors
* Naive Bayes
* Decision Tree
* Random Forest
* Logistic 
* XgBoost
* Support Vector Machine
* Neural Network

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# read file
voice=pd.read_csv('/kaggle/input/voicegender/voice.csv')
voice.head()

# Data Describing and Preprocessing

In [ ]:
voice.info()

In [ ]:
voice.describe()

In [ ]:
# Converting male/female to 1/0
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
voice["label"] = le.fit_transform(voice["label"])
le.classes_

In [ ]:
# Transforming num columns with MinMax Scale
voice[:]=preprocessing.MinMaxScaler().fit_transform(voice)
voice.head()

## Data Exploration

In [ ]:
# Visualization voice data for exploring significant correlation between gender
import seaborn as sns
import matplotlib.pyplot as plt
plt.subplots(4,5,figsize=(20,20))
for i in range(1,21):
    plt.subplot(4,5,i)
    plt.title(voice.columns[i-1])
    sns.kdeplot(voice.loc[voice['label'] == 0, voice.columns[i-1]], color= 'green', label='F')
    sns.kdeplot(voice.loc[voice['label'] == 1, voice.columns[i-1]], color= 'blue', label='M')

We can choose Q25, IQR and meanfun columns for significant corr between gender. When we examine it, there is a determining average value in the values of these columns, which separates according to gender to form a double peak and is specific to that gender.

# Data Modeling

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
from sklearn import neural_network
import xgboost
from sklearn.linear_model import LogisticRegression

In [ ]:
# Split the data
train, test = train_test_split(voice, test_size=0.3)

In [ ]:
#Modeling for all data
x_train = train.iloc[:, :-1]
y_train = train["label"]
x_test = test.iloc[:, :-1]
y_test = test["label"]

In [ ]:
#Modeling for three columns
x_train3 = train[["meanfun","IQR","Q25"]]
y_train3 = train["label"]
x_test3 = test[["meanfun","IQR","Q25"]]
y_test3 = test["label"]

In [ ]:
#Function about model classifying report
def classify(model,x_train,y_train,x_test,y_test):
    from sklearn.metrics import classification_report
    target_names = ['female', 'male']
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    print(classification_report(y_test, y_pred, target_names=target_names, digits=4))

### M1 : K-Nearest Neighbors
Using neighbors.KNeighborsClassifier() to build the model.

In [ ]:
# KNN error report function for visualization
def knn_error(k,x_train,y_train,x_test,y_test):
    error_rate = []
    K=range(1,k)
    for i in K:
        knn = neighbors.KNeighborsClassifier(n_neighbors = i)
        knn.fit(x_train, y_train)
        y_pred = knn.predict(x_test)
        error_rate.append(np.mean(y_pred != y_test))
    kloc = error_rate.index(min(error_rate))
    print("Lowest error is %s occurs at k=%s." % (error_rate[kloc], K[kloc]))

    plt.plot(K, error_rate, color='blue', linestyle='dashed', marker='o',
             markerfacecolor='red', markersize=10)
    plt.title('Error Rate vs. K Value')
    plt.xlabel('K')
    plt.ylabel('Error Rate')
    plt.show()
    return K[kloc]

In [ ]:
#for all data
k=knn_error(21,x_train,y_train,x_test,y_test)

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors = k)
classify(model,x_train,y_train,x_test,y_test)

In [ ]:
#for 3 columns
k=knn_error(21,x_train3,y_train3,x_test3,y_test3)

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors = k)
classify(model,x_train3,y_train3,x_test3,y_test3)

### M2: Naive Bayes
Using naive_bayes.GaussianNB() to build the model.

In [ ]:
#for all data
model=naive_bayes.GaussianNB()
classify(model,x_train,y_train,x_test,y_test)

In [ ]:
#for 3 columns
model=naive_bayes.GaussianNB()
classify(model,x_train3,y_train3,x_test3,y_test3)

### M3: Decision Tree
Using tree.DecisionTreeClassifier() to build the model.

In [ ]:
#Find the best parameter to prune the tree
def dt_error(n,x_train,y_train,x_test,y_test):
    nodes = range(2, n)
    error_rate = []
    for k in nodes:
        model = tree.DecisionTreeClassifier(max_leaf_nodes=k)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        error_rate.append(np.mean(y_pred != y_test))
    kloc = error_rate.index(min(error_rate))
    print("Lowest error is %s occurs at n=%s." % (error_rate[kloc], nodes[kloc]))
    plt.plot(nodes, error_rate, color='blue', linestyle='dashed', marker='o',
             markerfacecolor='red', markersize=10)
    plt.xlabel('Tree Size')
    plt.ylabel('Cross-Validated MSE')
    plt.show()
    return nodes[kloc]

In [ ]:
n=dt_error(10,x_train,y_train,x_test,y_test)

In [ ]:
#prune tree
pruned_tree = tree.DecisionTreeClassifier(criterion = 'gini', max_leaf_nodes = n)
classify(pruned_tree,x_train,y_train,x_test,y_test)

In [ ]:
#for three columns
n=dt_error(15,x_train3,y_train3,x_test3,y_test3)

In [ ]:
#prune tree
pruned_tree = tree.DecisionTreeClassifier(criterion = 'gini', max_leaf_nodes = n)
classify(pruned_tree,x_train3,y_train3,x_test3,y_test3)

### M4: Random Forest
Using ensemble.RandomForestClassifier() to build the model.

In [ ]:
#Function for visualization of RF model error rate
def rf_error(n,x_train,y_train,x_test,y_test):
    error_rate = []
    e=range(1,n,20)
    for i in e:
        model = ensemble.RandomForestClassifier(n_estimators = i)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        error_rate.append(np.mean(y_pred != y_test))
    nloc = error_rate.index(min(error_rate))
    print("Lowest error is %s occurs at n=%s." % (error_rate[nloc], e[nloc]))

    plt.plot(e, error_rate, color='blue', linestyle='dashed', marker='o',
             markerfacecolor='red', markersize=10)
    plt.title('Error Rate vs. n Value')
    plt.xlabel('n')
    plt.ylabel('Error Rate')
    plt.show()
    return e[nloc]

In [ ]:
#All data
e=rf_error(100,x_train,y_train,x_test,y_test)

In [ ]:
model=ensemble.RandomForestClassifier(n_estimators = e)
classify(model,x_train,y_train,x_test,y_test)

In [ ]:
#for 3 columns
e=rf_error(100,x_train3,y_train3,x_test3,y_test3)

In [ ]:
model=ensemble.RandomForestClassifier(n_estimators = e)
classify(model,x_train3,y_train3,x_test3,y_test3)

### M5: XgBoost
Using xgboost.XGBClassifier() to build the model.

In [ ]:
#for all columns
model = xgboost.XGBClassifier()
classify(model,x_train,y_train,x_test,y_test)

In [ ]:
#for three columns
model = xgboost.XGBClassifier()
classify(model,x_train3,y_train3,x_test3,y_test3)

### M6: Support Vector Machine
Using svm.SVC() to build the model.

In [ ]:
# Function for highest accuracy
def svm_kernel(x_train,y_train,x_test,y_test):
    rate=[]
    kernel=['rbf','poly','linear']
    for i in kernel:
        model=svm.SVC(kernel=i).fit(x_train,y_train)
        y_pred=model.predict(x_train)
        print(i, ' in-sample accuracy in SVM: ', accuracy_score(y_train,y_pred))
        y_pred=model.predict(x_test)
        print(i, ' out-of-sample accuracy in SVM: ', accuracy_score(y_test,y_pred))
        rate.append(accuracy_score(y_test,y_pred))
    nloc = rate.index(max(rate))
    print("Highest accuracy is %s occurs at %s kernel." % (rate[nloc], kernel[nloc]))
    return kernel[nloc]

In [ ]:
#Function for lowest error
def svm_error(k,C,x_train,y_train,x_test,y_test):
    error_rate = []
    C=range(1,C)
    for i in C:
        model=svm.SVC(kernel=k,C=i).fit(x_train,y_train)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        error_rate.append(np.mean(y_pred != y_test))
    cloc = error_rate.index(min(error_rate))
    print("Lowest error is %s occurs at C=%s." % (error_rate[cloc], C[cloc]))

    plt.plot(C, error_rate, color='blue', linestyle='dashed', marker='o',
             markerfacecolor='red', markersize=10)
    plt.title('Error Rate vs. C Value')
    plt.xlabel('C')
    plt.ylabel('Error Rate')
    plt.show()
    return C[cloc]

In [ ]:
k=svm_kernel(x_train,y_train,x_test,y_test)

In [ ]:
c=svm_error(k,10,x_train,y_train,x_test,y_test)

In [ ]:
# with higest accuracy and lowest error SVm model for all columns
model=svm.SVC(kernel=k,C=c)
classify(model,x_train,y_train,x_test,y_test)

In [ ]:
# for 3 columns
k=svm_kernel(x_train3,y_train3,x_test3,y_test3)

In [ ]:
c=svm_error(k,10,x_train3,y_train3,x_test3,y_test3)

In [ ]:
# for highest accuracy and lowest error with 3 columns
model=svm.SVC(kernel=k,C=c)
classify(model,x_train3,y_train3,x_test3,y_test3)

### M7: Neural Network
Using neural_network.MLPClassifier to build the model.

In [ ]:
def nn_error(n,x_train,y_train,x_test,y_test):
    error_rate = []
    hidden_layer=range(1,n)
    for i in hidden_layer:
        model = neural_network.MLPClassifier(solver='adam', alpha=1e-5,
                                       hidden_layer_sizes=i,
                                       activation='logistic',random_state=17,
                                       max_iter=2000)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        error_rate.append(np.mean(y_pred != y_test))
    kloc = error_rate.index(min(error_rate))
    print("Lowest error is %s occurs at C=%s." % (error_rate[kloc], hidden_layer[kloc]))

    plt.plot(hidden_layer, error_rate, color='blue', linestyle='dashed', marker='o',
             markerfacecolor='red', markersize=10)
    plt.title('Error Rate vs. Hidden Layer Size')
    plt.xlabel('Size')
    plt.ylabel('Error Rate')
    plt.show()
    return hidden_layer[kloc]

In [ ]:
h=nn_error(20,x_train,y_train,x_test,y_test)

In [ ]:
model = neural_network.MLPClassifier(solver='adam', alpha=1e-5,
                                       hidden_layer_sizes=h,
                                       activation='logistic',random_state=17,
                                       max_iter=2000)
classify(model,x_train,y_train,x_test,y_test)

In [ ]:
# for three columns
h=nn_error(20,x_train3,y_train3,x_test3,y_test3)

In [ ]:
#for 3 columns train data
model = neural_network.MLPClassifier(solver='adam', alpha=1e-5,
                                       hidden_layer_sizes=h,
                                       activation='logistic',random_state=17,
                                       max_iter=2000)
classify(model,x_train3,y_train3,x_test3,y_test3)

## M8: Logistic Regression
Using LogisticRegression to build the model.

In [ ]:
model=LogisticRegression()
#modeling for all columns
classify(model,x_train,y_train,x_test,y_test)

In [ ]:
#modeling for 3 columns
classify(model,x_train3,y_train3,x_test3,y_test3)

We can see that the highest accurracy is 97.7% which is made by KNN.